In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

'''
    Author:	 Matthew Ogden and Graham West
    Created:	01 Sep 2020
Description:	This code is a modified version of code written by Graham West at https://github.com/gtw2i/GA-Galaxy. 
                The pupose of this code is to genetrically modify the SPAM orbital parameters for galactic models.
'''

# Add parent directory
from sys import path as sysPath
from os import listdir
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Genetic_Algorithm import main_Genetic_Algorithm as ga
import main_SIMR as simr

# General modules
from os import path
import numpy as np
import numpy.linalg as LA
import pandas as pd
from matplotlib import pyplot as plt
import random
import pickle
from time import sleep 

# Print testing
gm.test()
im.test()
ga.test()
simr.test()


GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
GA: Hi!  You're in Matthew's main code for all things genetric algorithm.
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [2]:
tDir = '../targetDir'
#print('Before: \'%s\' \nAfter: \'%s\'' % (tDir,gm.validPath(tDir) ))
tDir = gm.validPath(tDir)
rInfo = None

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")
    im.tabprint("Grabbing first model.")
    rInfo = tInfo.getRunInfo()
    #print(rInfo)
    if type(rInfo) == type(None):
        print("BAD RUN!!!!")
    else:
        print("RUN GOOOD!!!")
        im.tabprint('rInfo status: %s' % (rInfo.status) )

Target Good!
	 - Grabbing first model.
RUN GOOOD!!!
	 - rInfo status: True


In [ ]:

# Executable parameter setup
cmdArgs = gm.inArgClass()
cmdArgs.setArg('scoreParamLoc', '../param/test_param.json')
cmdArgs.setArg('gaParamLoc', '../param/init_ga_param.json')
cmdArgs.setArg('targetDir', '../targetDir')

cmd_dict = {}
cmd_dict['printAll'] = True
cmd_dict['gaExp'] = True
cmd_dict['workerLocName'] = 'target'

cmdArgs.updateArgsFromDict( cmd_dict )

simr.target_main( cmdArg = cmdArgs )

SIMR: target_main:
SIMR.target_initialize:
	 - tDir: ../targetDir
	 - tInfo: <class 'NoneType'>
IM: target_info_class.__init__:
	 - targetDir:  ../targetDir
	 - arg.targetDir:  ../targetDir
IM: Target.initTargetDir():
	 - targetDir: ../targetDir
	 - targetDir: (True) /home/mbo2d/587722984435351614/
	 - infoDir: (True) /home/mbo2d/587722984435351614/information/
	 - baseInfoLoc: (True) /home/mbo2d/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /home/mbo2d/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /home/mbo2d/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /home/mbo2d/587722984435351614/plots/
IM: Target: Opening target info json
	 - True - 587722984435351614 - 1293 Models
SIMR.GA_Experiment_Wrapper:
	 - Setting up parameters
SIMR: target_prep_cmd_params: 587722984435351614
	 - 0 in target_prep_cmd_params: 587722984435351614: 
SIMR.target_prep_cmd_params: Printing new run argments

GM: Printing Input arguments
	- print

In [ ]:


ga_param = Prep_GA_Input_Parameters('../param/init_ga_param.json')
print( ga_param.keys() )

In [ ]:
outDir = gm.validPath ( '../tmpDir/' )

outBase = outDir + 'Results_%s_' % gm.getFileFriendlyDateTime()

exp_details = {}

exp_details['name'] = 'Genetic Algorithm Creation'
exp_details['info'] = 'This file is the prototype for creating \
    the Genetic Algorythm pipeline.'
exp_details['score_parameters'] = runArgs.get('scoreParams')
exp_details['ga_paramters'] = ga_param

print( exp_details.keys() )

eLoc = outBase + 'Details.json'
print(eLoc)

gm.saveJson( exp_details, eLoc, pretty = False, convert_numpy_array=True )


print("Saved")

In [ ]:
Genetic_Algorithm_Experiment( ga_param, scorer.score_models, outBase, True )

In [ ]:
models = pickle.load( open( outBase + 'models.pkl', 'rb') )
print(models.shape)